<a href="https://colab.research.google.com/github/ijortin/CS-4485-Virtual-TA/blob/tokenizing/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paackage installation

In [ ]:
!pip install tflearn
!pip install nltk
!pip install numpy
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 9.2 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=2aa6fc2de27126fd855536f08d6f59f8201350cb50bed7afcecc420a12877972
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Imports

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from google.colab import files
uploaded = files.upload()
tempData = next(iter(uploaded.values()))

Saving intents.json to intents.json


Load in data

In [ ]:
data = json.loads(tempData.decode())

In [ ]:
try:
  with open("data.pickle", "rt") as f:
    words, labels, training, output = pickle.load(f)

except:
  words = []
  labels = []
  docs_x = []
  docs_y = []

  for intent in data["intents"]:
      for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

      if intent["tag"] not in labels:
        labels.append(intent["tag"])
 
  words = [stemmer.stem(w.lower()) for w in words if w != "?"]
  words = sorted(list(set(words)))

  labels = sorted(labels)

  training = []
  output = []

  out_empty = [0 for _ in range(len(labels))]

  for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

  training = numpy.array(training)
  output = numpy.array(output)

Train Model

In [ ]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 5)
net = tflearn.fully_connected(net, 6)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(training, output, n_epoch=100, batch_size=1, show_metric= True)
model.save("model.tflearn")

Training Step: 3399  | total loss: 0.27291 | time: 0.198s
| Adam | epoch: 100 | loss: 0.27291 - acc: 0.8612 -- iter: 33/34
Training Step: 3400  | total loss: 0.24570 | time: 0.205s
| Adam | epoch: 100 | loss: 0.24570 - acc: 0.8751 -- iter: 34/34
--


Predictions

In [ ]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1
  
  return numpy.array(bag)

In [ ]:
COMPLEXITY_COMPS = {
    'log(n)':1,
    ['n', '2n']:2,
    'nlog(n)':3,
    'n^2':4,
    '2^n':5,
    'n!':6
}

In [ ]:
def bigOComp(i_words):
    val1 = 0
    val2 = 0
    for word in i_words:
        if word in COMPLEXITY_COMPS.keys():
            if val1==0:
                val1 = word
                continue
            else:
                val2 = word
                break
    if COMPLEXITY_COMPS[val1] > COMPLEXITY_COMPS[val2]:
        print(f"O({val1}) is slower than O({val2})")
    else:
        print(f"O({val1}) is faster than O({val2})")


In [ ]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        responses = None

        for tg in data["intents"]:
            if tg['tag'] == 'Big-O':
              i_words = inp.split(" ")
              bigOComp(i_words)
            elif tg['tag'] == tag:
              responses = tg['responses']
        if responses!=None:
            print(random.choice(responses))

In [ ]:
chat()

Start talking with the bot (type quit to stop)!
You: is O( n ) faster than O( log(n) )
O(n) is slower than O(log(n))
